In [1]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .master("local[*]") \
    .getOrCreate()

# Access SparkContext from SparkSession
sc = spark.sparkContext

In [2]:
df= spark.read.format('csv').option('inferSchema',True).option('header',True).load('/users/shubh/Data Engineering/BigMart Sales.csv')

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
%config Completer.use_jedi = False

In [4]:
df.toPandas()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,None,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


# when otherwise

### Scenario 1

In [11]:
df= df.withColumn('Non_veg_flag', when(col('Item_Type')=='Meat','Non-veg').otherwise('Veg'))

In [12]:
df.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|Non_veg_flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|         Veg|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|              

In [14]:
df.withColumn('Veg_exp_flag',when((col('Non_veg_flag')=='Veg') & (col('Item_MRP') < 100), 'Veg_Inexpensive')\
                 .when((col('Non_veg_flag')=='Veg') & (col('Item_MRP') > 100), 'Veg_expensive')\
                 .otherwise('Non-Veg')).toPandas()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Non_veg_flag,Veg_exp_flag
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,Veg,Veg_expensive
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,Veg,Veg_Inexpensive
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,Non-veg,Non-Veg
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,Veg,Veg_expensive
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Veg,Veg_Inexpensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,Veg,Veg_expensive
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,None,Tier 2,Supermarket Type1,549.2850,Veg,Veg_expensive
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,Veg,Veg_Inexpensive
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976,Veg,Veg_expensive


# Joins

In [15]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')] 

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING' 

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [16]:
df1.show()
df2.show()

+------+--------+-------+
|emp_id|emp_name|dept_id|
+------+--------+-------+
|     1|    gaur|    d01|
|     2|     kit|    d02|
|     3|     sam|    d03|
|     4|     tim|    d03|
|     5|    aman|    d05|
|     6|     nad|    d06|
+------+--------+-------+

+-------+----------+
|dept_id|department|
+-------+----------+
|    d01|        HR|
|    d02| Marketing|
|    d03|  Accounts|
|    d04|        IT|
|    d05|   Finance|
+-------+----------+



### inner join

In [17]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').show()

+------+--------+-------+-------+----------+
|emp_id|emp_name|dept_id|dept_id|department|
+------+--------+-------+-------+----------+
|     1|    gaur|    d01|    d01|        HR|
|     2|     kit|    d02|    d02| Marketing|
|     3|     sam|    d03|    d03|  Accounts|
|     4|     tim|    d03|    d03|  Accounts|
|     5|    aman|    d05|    d05|   Finance|
+------+--------+-------+-------+----------+



### left join

In [19]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'left').show()

+------+--------+-------+-------+----------+
|emp_id|emp_name|dept_id|dept_id|department|
+------+--------+-------+-------+----------+
|     1|    gaur|    d01|    d01|        HR|
|     2|     kit|    d02|    d02| Marketing|
|     3|     sam|    d03|    d03|  Accounts|
|     4|     tim|    d03|    d03|  Accounts|
|     5|    aman|    d05|    d05|   Finance|
|     6|     nad|    d06|   NULL|      NULL|
+------+--------+-------+-------+----------+



# right join

In [20]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'right').show()

+------+--------+-------+-------+----------+
|emp_id|emp_name|dept_id|dept_id|department|
+------+--------+-------+-------+----------+
|     1|    gaur|    d01|    d01|        HR|
|     2|     kit|    d02|    d02| Marketing|
|     4|     tim|    d03|    d03|  Accounts|
|     3|     sam|    d03|    d03|  Accounts|
|  NULL|    NULL|   NULL|    d04|        IT|
|     5|    aman|    d05|    d05|   Finance|
+------+--------+-------+-------+----------+



### anti join

In [21]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'anti').show()

+------+--------+-------+
|emp_id|emp_name|dept_id|
+------+--------+-------+
|     6|     nad|    d06|
+------+--------+-------+



In [22]:
df2.join(df1, df2['dept_id']==df1['dept_id'],'anti').show()

+-------+----------+
|dept_id|department|
+-------+----------+
|    d04|        IT|
+-------+----------+



# Window functions

### RowNumber()

In [6]:
from pyspark.sql.window import Window

In [24]:
df.withColumn('RowNumber',row_number().over(Window.orderBy('Item_Identifier'))).toPandas()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Non_veg_flag,RowNumber
0,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,None,Tier 2,Supermarket Type1,2552.6772,Veg,1
1,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,None,Tier 2,Supermarket Type1,3829.0158,Veg,2
2,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772,Veg,3
3,DRA12,11.60,LF,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078,Veg,4
4,DRA12,11.60,Low Fat,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,850.8924,Veg,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,NCZ54,14.65,Low Fat,0.083528,Household,162.4552,OUT045,2002,None,Tier 2,Supermarket Type1,2274.3728,Veg,8519
8519,NCZ54,14.65,Low Fat,0.083359,Household,161.9552,OUT046,1997,Small,Tier 1,Supermarket Type1,4711.2008,Veg,8520
8520,NCZ54,NaN,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,Grocery Store,162.4552,Veg,8521
8521,NCZ54,14.65,Low Fat,0.083699,Household,163.4552,OUT018,2009,Medium,Tier 3,Supermarket Type2,2599.2832,Veg,8522


### Rank


In [26]:
df.withColumn('Rank',rank().over(Window.orderBy('Item_Identifier'))).show()

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|Non_veg_flag|Rank|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----+
|          DRA12|       11.6|         Low Fat|    0.041177505|Soft Drinks|140.3154|           OUT017|                     2007|       NULL|              Tier 2|Supermarket Type1|        2552.6772|         Veg|   1|
|          DRA12|       11.6|         Low Fat|            0.0|Soft Drinks|141.6154|           OUT045|                     2002|       NULL| 

### DenseRank

In [27]:
df.withColumn('DenseRank', dense_rank().over(Window.orderBy('Item_Identifier'))).show()

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|Non_veg_flag|DenseRank|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+---------+
|          DRA12|       11.6|         Low Fat|    0.041177505|Soft Drinks|140.3154|           OUT017|                     2007|       NULL|              Tier 2|Supermarket Type1|        2552.6772|         Veg|        1|
|          DRA12|       11.6|         Low Fat|            0.0|Soft Drinks|141.6154|           OUT045|                   

In [29]:
df.withColumn('Rank',rank().over(Window.orderBy('Item_Identifier')))\
    .withColumn('DenseRank', dense_rank().over(Window.orderBy('Item_Identifier'))).show()  #combination of rank and DenseRank

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----+---------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|Non_veg_flag|Rank|DenseRank|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----+---------+
|          DRA12|       11.6|         Low Fat|    0.041177505|Soft Drinks|140.3154|           OUT017|                     2007|       NULL|              Tier 2|Supermarket Type1|        2552.6772|         Veg|   1|        1|
|          DRA12|       11.6|         Low Fat|            0.0|Soft Drinks|141.6154|           OUT045

# cumilative sum

In [9]:
df.withColumn('Cumsum', sum('Item_MRP').over(Window.orderBy('Item_Type'))).toPandas()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Cumsum
0,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,8.189474e+04
1,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432,8.189474e+04
2,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876,8.189474e+04
3,FDL12,15.850,Regular,0.121633,Baking Goods,60.6220,OUT046,1997,Small,Tier 1,Supermarket Type1,2576.6460,8.189474e+04
4,FDL12,15.850,Regular,0.121532,Baking Goods,59.2220,OUT013,1987,High,Tier 3,Supermarket Type1,599.2200,8.189474e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDA34,11.500,Low Fat,0.014884,Starchy Foods,174.5080,OUT049,1999,Medium,Tier 1,Supermarket Type1,3635.2680,1.201681e+06
8519,FDC59,16.700,Regular,0.054851,Starchy Foods,64.4168,OUT018,2009,Medium,Tier 3,Supermarket Type2,830.9184,1.201681e+06
8520,FDD47,7.600,Regular,0.142632,Starchy Foods,172.3448,OUT049,1999,Medium,Tier 1,Supermarket Type1,4431.5648,1.201681e+06
8521,FDA22,7.435,Low Fat,0.084436,Starchy Foods,168.6158,OUT035,2004,Small,Tier 2,Supermarket Type1,4512.1266,1.201681e+06


In [11]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type')\
                                            .rowsBetween(Window.unboundedPreceding,Window.currentRow))).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|            cumsum|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------------+
|          FDP36|     10.395|         Regular|            0.0|Baking Goods| 51.4008|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         556.6088|           51.4008|
|          FDW12|       NULL|         Regular|    0.035399923|Baking Goods|144.5444|           OUT027|                     1985|    

In [12]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type')\
                                            .rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).show()

+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|   Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|            cumsum|
+---------------+-----------+----------------+---------------+------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------------+
|          FDP36|     10.395|         Regular|            0.0|Baking Goods| 51.4008|           OUT018|                     2009|     Medium|              Tier 3|Supermarket Type2|         556.6088|1201681.4808000019|
|          FDW12|       NULL|         Regular|    0.035399923|Baking Goods|144.5444|           OUT027|                     1985|    

# User defined functions(UDF)

### Step1

In [17]:
def newFunc(x):
    return x*x

In [21]:
my_udf= udf(newFunc)

### Step2

In [19]:
df.withColumn('SquareMRP',my_udf('Item_MRP')).show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|         SquareMRP|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|62404.636404640005|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|        

# Data Writing

### csv

In [6]:
df.write.format('csv')\
        .save('/users/shubh/BigData_Data/Data.csv')

#### APPEND

In [7]:
df.write.format('csv')\
        .mode('append')\
        .save('/users/shubh/BigData_Data/Data.csv')

In [10]:
df.write.format('csv')\
        .mode('append')\
        .option('path','/users/shubh/BigData_Data/Data.csv')\
        .save()

#### Overwrite

In [11]:
df.write.format('csv')\
.mode('overwrite')\
.option('path','/users/shubh/BigData_Data/Data.csv')\
.save()

#### Error

In [13]:
df.write.format('csv')\
.mode('error')\
.option('path','/users/shubh/BigData_Data/Data.csv')\
.save()


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/users/shubh/BigData_Data/Data.csv already exists. Set mode as "overwrite" to overwrite the existing path. SQLSTATE: 42K04

#### PARQUET

In [14]:
df.write.format('parquet')\
.mode('overwrite')\
.option('path','/users/shubh/BigData_Data/Data.csv')\
.save()

#### TABLE

In [15]:
df.write.format('parquet')\
.mode('overwrite')\
.saveAsTable('my_table')

In [23]:
df.toPandas()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,None,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


# SQL view

In [17]:
df.createTempView('my_view')

In [24]:
result_all = spark.sql("select * from my_view where Item_Fat_Content = 'Low Fat'")
result_all.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          FDN15|       17.5|         Low Fat|    0.016760075|                Meat| 141.618|           OUT049|                     1999|     Medium|              Tier 1|Superma

# End
